In [15]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%autoreload 2

In [141]:
from Core.Nanoparticle import Nanoparticle

p = Nanoparticle()
p.truncated_octahedron(6,1,{'Pt':0.5,'Au':0.5})

p.construct_adsorption_list()
p.random_occupation(10)
ads = AdsorptionFeatureVector(symbols)
ads.compute_feature_vector(p)
p.get_feature_vector('ADS')


Resulting stoichiometry: {'Au': 70, 'Pt': 70}


array([1, 1, 1, 3, 0, 0, 1, 3, 0])

In [154]:
def random_ex():
    status0_indices = np.random.choice(p.get_occupation_status_by_indices(0), 1, replace=False)
    status1_indices = np.random.choice(p.get_occupation_status_by_indices(1), 1, replace=False)
    p.swap_status(zip(status0_indices, status1_indices))

In [145]:
ads.compute_feature_vector(p)
p.get_feature_vector('ADS')

array([0, 1, 1, 3, 0, 0, 1, 4, 0])

In [149]:
Au = 0
Ni = 0
Au2 = -0.28120172 
Au2Ni = -1.00039431 
Au3 = -0.35726243
AuNi = -1.01410455
AuNi2 = -2.08995281
Ni2 = -2.23349599
Ni3 = -2.40469383

In [166]:
from Core.GlobalFeatureClassifier import AdsorptionFeatureVector
from Core.EnergyCalculator import BayesianRRCalculator

energy_calculator = BayesianRRCalculator('ADS')
sergey_coefficients = [Au, Ni, Au2, AuNi, Ni2, Au3, Au2Ni, AuNi2, Ni3]
energy_calculator.set_coefficients(sergey_coefficients)
ads = AdsorptionFeatureVector(symbols)
random_ex()
ads.compute_feature_vector(p)
p.get_feature_vector('ADS')
energy_calculator.compute_energy(p)
print(p.get_energy('BRR'), p.get_number_of_adsorbates())


-11.156317430000001 10


In [188]:
from MCMC import MonteCarlo as MC

start_particle = Nanoparticle()
start_particle.truncated_octahedron(6,1,{'Au':0.5,'Ni':0.5})

p.construct_adsorption_list()

beta = 200
max_steps = 500
energy_calculator = BayesianRRCalculator('ADS')
sergey_coefficients = [Au, Ni, Au2, AuNi, Ni2, Au3, Au2Ni, AuNi2, Ni3]
energy_calculator.set_coefficients(sergey_coefficients)
n_adsorbates = 10

[best_particle, accepted_energies] = MC.run_monte_carlo_for_adsorbates(beta, max_steps, start_particle, energy_calculator, n_adsorbates)

Resulting stoichiometry: {'Au': 70, 'Ni': 70}


/home/riccardo/Projects/NPL/MCMC/MonteCarlo.py:204: RuntimeWarning: overflow encountered in exp
  if np.random.random() < acceptance_rate:


In [187]:
start_particle.get_site_atom_indices(518)

{111, 112, 113}

In [218]:
from ase.visualize import view
from Core.Adsorption import PlaceAddAtoms

best_p = Nanoparticle()
best_p.build_from_dictionary(best_particle)
best_p.construct_neighbor_list()
best_p.construct_adsorption_list()
ads = PlaceAddAtoms(best_p.get_all_symbols())
ads.bind_particle(best_p)
for f in best_particle['ads']:
    site = list(best_p.get_site_atom_indices(f))
    ads.place_add_atom(best_p, 'O', [site])

#view(best_p.get_ase_atoms(), viewer='ngl')

In [217]:
f

190